In [ ]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

# Trabajo Práctico 2: Entrenamiento y evaluación de modelos
---

## Fecha y hora de entrega máxima:
09/05/2022 18:00

## Dataset "Datos de clientes del banco"
Los datos están relacionados con campañas de marketing directo (llamadas telefónicas) de una institución bancaria portuguesa. El objetivo de la clasificación es predecir si el cliente suscribirá un depósito a plazo.

<img src="https://storage.googleapis.com/kaggle-datasets-images/864595/1473402/1f559c7d6d646d0a5f24c1847fb10225/dataset-cover.jpg?t=2020-09-08-19-15-14"></img>

In [ ]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import pandas as pd
import sklearn_pandas
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, QuantileTransformer
from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from collections import defaultdict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

## **IMPORTANDO DATASET**

In [ ]:
# To replace these values with NaN, we must provide a list with all missing value formats
missing_value_formats = ["unknown", "n.a.","?","NA","n/a", "na", "--"]
dataset_original = pd.read_csv("BankCustomerData.csv", na_values = missing_value_formats)
# We will fill all NaN values with a string with value 'missing'
dataset_original.fillna('missing', inplace=True)
ds = dataset_original
ds_feature_eng = dataset_original
ds

## **FEATURE ENGINEERING**

Nos parecio conveniente avanzar sin utilizar técnicas de feature engineering ya que entendemos que los datos del DS son lo suficientemente representativos. No vemos conveniente aplicar redondeos ya que las variables numéricas que tenemos (age, balance, duration) son de tipo entero. Tampoco observamos columnas que contengan información "escondida" que convenga extraerla para crear nuevas features.
Por otro lado si hemos utilizado técnicas de pre-procesamiento como por ejemplo StandardScaller

### **Train, Validation and Test**

In [ ]:
ds["term_deposit"] = ds.term_deposit.replace(['no', 'yes'], [0,1])
#ds_feature_eng["term_deposit"] = ds_feature_eng.term_deposit.replace(['no', 'yes'], [0,1])
# Dividimos el dataset en train (60%), test (20%) y validation (20%)
train, not_train = train_test_split(ds, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

## **MÉTRICA A UTILIZAR**
La métrica que utilizaremos es **Precission**, ya que, errar por si o por no tiene diferente impacto en este problema. Nos interesa tener cierta seguridad de que a la hora de llamar a un cliente para ofrecerle un depósito a plazo, exista un alto porcentaje de que este quiera avanzar en el proceso. De esta manera evitamos generar cierto rechazo y mal estar del cliente hacia la organización.

Por otra parte, también utilizaremos la métrica **Accuracy**, la cual mide el porcentaje de casos que el modelo ha acertado. Esta es una de las métricas más usadas, es sencillo de explicar al usuario final que requiere el modelo, y es oportuna para casos de clasificación. 

Sin embargo nos puede llevar al engaño, es decir, puede hacer que un modelo malo parezca que es mucho mejor de lo que realmente es, es por ésto que también tenemos a **Precission**.

## **MAPPING DE VARIABLES**

In [ ]:
mapper = DataFrameMapper([
    (['age'],[StandardScaler()]),
    (['loan'],[OneHotEncoder()]),
    (['housing'],[OneHotEncoder()]),
    (['job'],[OneHotEncoder()]),
    (['education'],[OneHotEncoder()]),
    (['balance'],[StandardScaler()])
])
mapper.fit(train)
mapper.transform(train)

## ENTRENAMIENTO DE MODELOS
Elegimos los siguientes 6 modelos para entrenar:
- Logistic Regression
- Arbol de Decisión
- Random Forest
- Gradient Boosting Classifier
- KNN
- Neural Networks MLP

In [ ]:
def evaluate_model(model, set_names=('train', 'validation'), title='', is_feature_engineering=False, show_confusion_matrix=True):
    if title:
        display(title)
    metrics_to_show = defaultdict(list)
    if show_confusion_matrix:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    for i, set_name in enumerate(set_names):
        if is_feature_engineering:
          assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]
        y = set_data.term_deposit
        y_pred = model.predict(set_data)
        metrics_to_show['Accuracy'].append(accuracy_score(y, y_pred))
        metrics_to_show['Precision'].append(precision_score(y, y_pred))
        if show_confusion_matrix:
            ax = axis[i]
            sns.heatmap(confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['No se suscribe', 'Se suscribe'])
            ax.yaxis.set_ticklabels(['No se suscribe', 'Se suscribe'])
            ax.set_xlabel('Clase Predecida')
            ax.set_ylabel('Clase Original')

    display(pd.DataFrame(metrics_to_show, index=set_names))
    if show_confusion_matrix:
        plt.tight_layout()
        plt.show()

### **Logistic Regression**

In [ ]:
model_logistic_regression = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=100)),
])
model_logistic_regression.fit(train, train.term_deposit)
evaluate_model(model_logistic_regression, title='Logistic Regression')

Podemos ver que tenemos un ratio de clasificación del 90.8% en train, y del 90.7 en validation, considerado como **buen accuracy.**

En cuanto a **precission**, podemos ver que es del 0%, con lo cual, o tiene datos que no "contienen" suficiente información para predecir la clase y el clasificador simplemente "adivina" la clase más frecuente. O bien tenemos muchos más datos de clase 1 que de clase 0 que la precisión es mejor si siempre adivina 1 en lugar de tratar de clasificar correctamente (como tenemos más term_deposit="no" que "yes", adivina siempre "no").

Cosas que podríamos hacer: 
1. Intentar obtener más datos de clase "si" (term_deposit="yes") de algún lugar.
2. Probar con otro clasificador que encaje mejor que la regresión logística.
3. Intentar obtener más y mejores datos, de diferentes fuentes, etc.

### **Arbol de Decisión**

In [ ]:
model_tree_decision = Pipeline([
    ('mapper', mapper),
    ('classifier', DecisionTreeClassifier(max_depth = 7, random_state=100)),
])
model_tree_decision.fit(train, train.term_deposit)
evaluate_model(model_tree_decision, title='Arbol de Decisión')

Podemos ver que tenemos un ratio de clasificación del 90.9% en train, y del 90.6 en validation, considerado como **buen accuracy.**

En cuanto a **precission**, de todas las veces que predice un resultado, el 78% de las veces está en lo correcto en train, y el 38% de las veces está en lo correcto en validation.

### **Random Forest**

In [ ]:
model_random_forest = Pipeline([
    ('mapper', mapper),
    ('classifier', RandomForestClassifier(random_state=100)),
])
model_random_forest.fit(train, train.term_deposit)
evaluate_model(model_random_forest, title='Random Forest')

Hay overfitting, para esto modificamos los hyperparametros dandole más profundidad

In [ ]:
model_random_forest_modified = Pipeline([
    ('mapper', mapper),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=15, max_features=15, random_state=100)),
])
model_random_forest_modified.fit(train, train.term_deposit)
evaluate_model(model_random_forest_modified, title='Random Forest with Depth = 15')

### **Gradient Boosting Classifier**

In [ ]:
model_gradient_boosting = Pipeline([
    ('mapper', mapper),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])
model_gradient_boosting.fit(train, train.term_deposit)
evaluate_model(model_gradient_boosting, title='Gradient Boosting Classifier')

Podemos ver que tenemos un ratio de clasificación del 91% en train, y del 90.7 en validation, considerado como **buen accuracy.**

En cuanto a **precission**, de todas las veces que predice un resultado, el 94% de las veces está en lo correcto en train, y el 40% de las veces está en lo correcto en validation.

### **KNN**

In [ ]:
model_knn = Pipeline([
    ('mapper', mapper),
    ('classifier', KNeighborsClassifier(n_neighbors=3)),
])
model_knn.fit(train, train.term_deposit)
evaluate_model(model_knn, title='K Nearest Neighbors with K = 6')

In [ ]:
model_knn = Pipeline([
    ('mapper', mapper),
    ('classifier', KNeighborsClassifier(n_neighbors=10)),
])
model_knn.fit(train, train.term_deposit)
evaluate_model(model_knn, title='K Nearest Neighbors with K = 10')

### **Neural Network MLP**

In [ ]:
model_mlp_classifier = Pipeline([
    ('mapper', mapper),
    ('classifier', MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(20,10),max_iter=20, random_state=42)),
])
model_mlp_classifier.fit(train, train.term_deposit)
evaluate_model(model_mlp_classifier, title='MLP Classifier')

Con modelo de MLP Classifier se obtiene un resultado bastante bueno aunque no mejor que los demás modelos. Tambien vemos que el mismo no mejora mas aumentando la cantidad de capas ocultas o la cantidad de iteraciones.

## OVERFITTING

In [ ]:
def generate_curve(selected_model="DecisionTree", list_to_iterate=list(range(1, 17))):
  train_prediction =  []
  eval_prediction = []
  for x in list_to_iterate:
    models = {
                "DecisionTree": Pipeline([
                  ('mapper', mapper),
                  ('classifier', DecisionTreeClassifier(max_depth=x, random_state=100)),
                ]),
                "RandomForest": Pipeline([
                  ('mapper', mapper),
                  ('classifier', RandomForestClassifier(n_estimators=100, max_depth=x, max_features=15, random_state=100)),
                ]),
                "KNN": Pipeline([
                  ('mapper', mapper),
                  ('classifier', KNeighborsClassifier(n_neighbors=x)),
                ]),
                "MLPClassifier": Pipeline([
                  ('mapper', mapper),
                  ('classifier', MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(20,10),max_iter=x, random_state=42)),
                ]),
                "GradientBoosting": Pipeline([
                    ('mapper', mapper),
                    ('classifier', GradientBoostingClassifier(learning_rate=x, max_depth=5, max_features=15, random_state=100)),
                ])
              }
    model = models[selected_model]
    model=model.fit(train, train.term_deposit)    
    train_prediction.append(model.score(train, train.term_deposit))
    eval_prediction.append(model.score(validation, validation.term_deposit))
  plt.plot(list_to_iterate, train_prediction, color='r', label='Train')
  plt.plot(list_to_iterate, eval_prediction, color='b', label='Validation')
  plt.title('Grafico ' + selected_model)
  plt.legend()
  plt.ylabel('Precisión')
  plt.xlabel('Cantidad de vecinos')
  plt.show()

### Curva para Árbol de decisión

In [ ]:
generate_curve("DecisionTree")

Como podemos ver en el gráfico anterior, entre una profundidad de 2 a 8 se encuentra el valor más eficiente para el algoritmo KNN. Luego de ésto podemos ver como ambas líneas divergen hacia los extremos, en el caso de train overfitea hacia el 100% y en el caso de validation deja de aprender, y cae hacia el 0%.

### Curva para Random Forest

In [ ]:
generate_curve("RandomForest")

### Curva para Gradient Boosting

In [ ]:
generate_curve("GradientBoosting")

Como podemos ver en el gráfico anterior, tenemos picos de precisión asignando un learning rate de 1, 6, 8 y 15, encontrando así en éstos puntos el valor más eficiente para el algoritmo Gradient Boosting. Luego de ésto podemos ver como ambas líneas tienen valles en el resto de números.

### Curva para KNN

In [ ]:
generate_curve("KNN")

Como podemos ver en el gráfico anterior, entre un K de 4 a 8 se encuentra el valor más eficiente para el algoritmo KNN. Luego de ésto podemos ver como ambas líneas convergen hacia el 0.91 aproximadamente, es decir se estancan y deja de aprender.

### Curva para MLPClassifier

In [ ]:
generate_curve("MLPClassifier")

Vemos que el modelo MLP Classifier no esta overfitiando ya que la métrica seleccionada no tiene grandes cambios entre Train y Validation y tambien viendo que la curva  de aprendizaje no varia en las primeras 10 iteraciones. 

## COMPARACIÓN FINAL ENTRE LOS DISTINTOS MODELOS
Comparación entre:
- Logistic Regression
- Arbol de Decisión
- Random Forest
- Gradient Boosting Classifier
- KNN
- Neural Networks MLP

In [ ]:
evaluate_model(model_logistic_regression, title='Logistic Regression', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)
evaluate_model(model_tree_decision, title='Arbol de Decisión', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)
evaluate_model(model_random_forest_modified, title='Random Forest with Depth = 15', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)
evaluate_model(model_gradient_boosting, title='Gradient Boosting Classifier', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)
evaluate_model(model_knn, title='K Nearest Neighbors with K = 6', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)
evaluate_model(model_mlp_classifier, title='MLPClassifier', set_names=('train', 'validation', 'test'), is_feature_engineering=False, show_confusion_matrix=False)

A raíz de los resultados obtenidos en los distintos modelos en relación a la métrica precission observamos que en general no obtuvimos buenos resultados logrando con Random Forest un porcentaje aceptable, aunque con este mismo vemos que el modelo overfitea bastante.
Con Gradient Boosting vemos un resultado similar de 0.52 aproximadamente y al igual que Random Forest la diferencia entre el dataset de train y los dataset de validación es notoria.
Con el modelo de redes neuronales Multi Layer Perceptron los resultados no son tan malos y aunque luego de modificar los hyperparametros fue mejorando pero no llegamos a lograr los resultados esperados.
Finalmente optamos por el Gradient Boosting ya que en promedio entre test y validation nos pareció mas eficiente.